In [1]:
maf_file_path = 'data/RESPOND_247_coding_final.maf'
expression_file_path = 'data/Expression_remove_BE.txt'
target_gene='RPL10'

import pandas as pd
import numpy as np
from statistics import mean 
from scipy.stats import ranksums
import matplotlib.pylab as plt
from statsmodels.stats import multitest


In [2]:
def load_maf_data(file_path):
    # Define columns of interest
    columns = ["Hugo_Symbol", "Tumor_Sample_Barcode"]
    df = pd.read_csv(file_path, sep='\t', comment="#", usecols=columns)
    df.rename(columns={'Hugo_Symbol': 'gene', 'Tumor_Sample_Barcode': 'sample'}, inplace=True)
    df['mutation'] = 1
    return df
    
maf_df = load_maf_data(maf_file_path)



In [3]:
def log2(value):
    return np.log2(value + 1)

def load_txt_file_into_dataframe(file_path):
    # Read the .txt file into a pandas DataFrame
    df = pd.read_csv(file_path, sep='\t')  # Adjust the separator if needed

    # take log2 of expression data to scale expression data
    df = df.map(log2)

    return df


# Call the load_txt_file_into_dataframe function
expression_df = load_txt_file_into_dataframe(expression_file_path)

In [4]:
def reformat_expression_data(df):
    # Combine column names and index names into rows for every element
    melted_df = pd.melt(df.reset_index(), id_vars=['index'], var_name='column', col_level=0)

    # Rename columns
    melted_df.rename(columns={'index': 'gene', 'column': 'sample', 'value': 'gene_expression'}, inplace=True)

    return melted_df


expression_df_melted = reformat_expression_data(expression_df)

In [5]:
# note individuals with muttated data mmay be missing in the expression data and vice versa. 
# every sample should have at least one mutation 

In [6]:
def preprocess_and_combine_mutation_expression(maf_df, expression_df):
    ''' filter the expression data to those that have whole genome sequencing 
     i.e. appear in the mutation data frame (maf)
     join mutation and expression data
     '''
    
    exon_seq_samples = maf_df['sample'].unique()
    filtered_expression_df = expression_df[expression_df['sample'].isin(exon_seq_samples)]
    
    all_rows = expression_df.shape[0]
    filt_rows = filtered_expression_df.shape[0]
    
    percentage_filtered = (all_rows - filt_rows) / all_rows
    
    print('fraction of rows filtered is', percentage_filtered)

    express_mut_genes_df = pd.merge(maf_df, filtered_expression_df, on=['gene', 'sample'], how='right')

    express_mut_genes_df['mutation'].fillna(0, inplace=True)

    return express_mut_genes_df


mutation_expression_df_melted =  preprocess_and_combine_mutation_expression(maf_df, expression_df_melted)
    

fraction of rows filtered is 0.2865853658536585


In [88]:
def calculate_log_fold(list_mutated, list_non_mutated, smoothing_factor=1e-8): 
    mean_mutated = mean(list_mutated)  + smoothing_factor
    mean_non_mutated = mean(list_non_mutated) + smoothing_factor

    # if np.isclose(mean_non_mutated, 0):
    #     print(f'the denominator is zero in log2 with a value of {mean_non_mutated}')
    #     return np.nan
        
    return np.log2(mean_mutated/mean_non_mutated)

In [50]:
def calculate_z_score(list_mutated, list_non_mutated):
    mean_mutated = mean(list_mutated) 
    mean_non_mutated = mean(list_non_mutated) 
    all_expression_values = list_non_mutated + list_mutated
    z_score = (mean_mutated - mean_non_mutated) / np.std(all_expression_values)
    return z_score


In [64]:
def calculate_pvalue_and_effect_size_wilcox_ranksum(list_mutated, list_non_mutated):
     test = ranksums(list_mutated, list_non_mutated)
     u_statistic = test.statistic 
     effect_size = u_statistic / (len(list_mutated) * len(list_non_mutated))
     pvalue=test.pvalue
     return pvalue, effect_size

In [63]:
def calculate_adjusted_pvalue(pvalues, method='fdr_bh'):
    _, corrected_pvalues, _, _ = multitest.multipletests(pvalues, method=method)
    return corrected_pvalues


In [101]:
import os
def generate_stats_per_gene(express_mut_genes_df, target_gene, output_folder=target_gene):
    """
    Given expression and mutation data calculates LogFC, pvalue, mean of expression per gene 
    for a given target_gene 
    
    """
    gene_df = express_mut_genes_df[express_mut_genes_df['gene']==target_gene]

    mutated_samples = gene_df[gene_df['mutation'] == 1]['sample']
    non_mutated_samples = gene_df[gene_df['mutation'] == 0]['sample']
    
    mutated_individuals_expression = express_mut_genes_df[express_mut_genes_df['sample'].isin(mutated_samples)]
    
    non_mutated_individuals_expression = express_mut_genes_df[express_mut_genes_df['sample'].isin(non_mutated_samples)]

    # gather data of mutated and non-mutated genes into lists
    mutated_individuals_data = mutated_individuals_expression.groupby(['gene'])['gene_expression'].apply(
    lambda x: list(x)).to_frame().reset_index().rename(columns={'gene_expression': 'gene_expression_mutated'})
    non_mutated_individuals_data = non_mutated_individuals_expression.groupby(['gene'])['gene_expression'].apply(
    lambda x: list(x)).to_frame().reset_index().rename(columns={'gene_expression': 'gene_expression_non_mutated'})

    # combine mutated and unmutated data into one df
    combined_data= pd.merge(mutated_individuals_data, non_mutated_individuals_data, on='gene', how='inner')

    # calculate fold change and p-value and z-score
    combined_data['logFC']= combined_data.apply(
        lambda x: calculate_log_fold(x.gene_expression_mutated, x.gene_expression_non_mutated), axis=1)
    # Assuming combined_data is a DataFrame
    combined_data[['pvalue', 'effect_size']] = combined_data.apply(
    lambda x: pd.Series(calculate_pvalue_and_effect_size_wilcox_ranksum(x.gene_expression_mutated, x.gene_expression_non_mutated)),
    axis=1
    )
    combined_data['expression_mutated_mean'] = combined_data['gene_expression_mutated'].apply(mean)
    combined_data['expression_nonmutated_mean'] = combined_data['gene_expression_non_mutated'].apply(mean)
 
    # Clip outliers in 'logFC' column
    # combined_data['logFC'] = combined_data['logFC'].clip(lower=-10, upper=10)

    # calculate adjusted p-value
    combined_data['adjusted_pvalue'] = calculate_adjusted_pvalue(combined_data['pvalue'].values)

    # Output data to csv 
    combined_data.drop(columns=['gene_expression_mutated', 'gene_expression_non_mutated'], inplace=True)
    
    os.makedirs(output_folder, exist_ok=True)
    output_filename = f'{output_folder}/{mutated_samples.count()}_{non_mutated_samples.count()}_logfc_pvalue.csv'
    
    print(f"outputting data to {output_filename}")
    combined_data.to_csv(output_filename, index=False)

    return combined_data, mutated_samples, output_filename


In [102]:
volcano_plot_df, individuals_mutated_target_gene, volcano_input_filename = generate_stats_per_gene(
    mutation_expression_df_melted, 
    target_gene)

outputting data to RPL10/5_112_logfc_pvalue.csv


In [103]:
volcano_plot_df

,gene,logFC,pvalue,effect_size,expression_mutated_mean,expression_nonmutated_mean,adjusted_pvalue
0,A1BG,-0.357606,0.526496,-0.001131,1.796602,2.301983,0.996941
1,A1CF,0.419188,0.589864,0.000963,0.770841,0.576470,0.996941
2,A2M,-0.058985,0.496169,-0.001215,7.180720,7.480391,0.996941
3,A2ML1,0.544969,0.140050,0.002635,1.805403,1.237434,0.996941
4,A3GALT2,0.183454,0.761733,0.000541,0.331263,0.291708,0.996941
...,...,...,...,...,...,...,...
19996,ZXDC,0.025356,0.898133,0.000229,5.923805,5.820600,0.996941
19997,ZYG11A,0.647638,0.035533,0.003754,1.981378,1.264763,0.996941
19998,ZYG11B,0.145653,0.188882,0.002346,3.902844,3.528054,0.996941
19999,ZYX,-0.133048,0.097413,-0.002960,7.000198,7.676475,0.996941


In [ ]:
volcano_plot_df['effect_size'].max()

In [87]:
volcano_plot_df.head(4)

,gene,logFC,pvalue,effect_size,expression_mutated_mean,expression_nonmutated_mean,adjusted_pvalue
0,A1BG,-0.357606,0.526496,-0.001131,1.796602,2.301983,0.996941
1,A1CF,0.419188,0.589864,0.000963,0.770841,0.576470,0.996941
2,A2M,-0.058985,0.496169,-0.001215,7.180720,7.480391,0.996941
3,A2ML1,0.544969,0.140050,0.002635,1.805403,1.237434,0.996941


In [104]:
volcano_plot_df['abs_logFC'] = volcano_plot_df['logFC'].abs()
volcano_plot_df.nlargest(50, 'abs_logFC')


,gene,logFC,pvalue,effect_size,expression_mutated_mean,expression_nonmutated_mean,adjusted_pvalue,abs_logFC
14559,RNU6-602P,26.651377,0.450461,0.001348,1.054056,0.000000,0.996941,26.651377
10679,NPB,-25.721730,0.129511,-0.002707,0.000000,0.553366,0.996941,25.721730
18936,VTRNA1-1,-25.495009,0.736195,-0.000602,0.000000,0.472893,0.996941,25.495009
10836,NT5DC4,-25.263995,0.113328,-0.002828,0.000000,0.402921,0.996941,25.263995
14485,RNU4-40P,25.159152,0.450461,0.001348,0.374679,0.000000,0.996941,25.159152
2423,CCL24,-25.105041,0.121209,-0.002767,0.000000,0.360886,0.996941,25.105041
14569,RNU6-761P,24.986909,0.450461,0.001348,0.332513,0.000000,0.996941,24.986909
4253,DLK1,-24.837117,0.177792,-0.002406,0.000000,0.299721,0.996941,24.837117
3677,CTAG2,-24.836412,0.438422,-0.001384,0.000000,0.299575,0.996941,24.836412
8447,KRTAP20-3,-24.827145,0.312166,-0.001805,0.000000,0.297656,0.996941,24.827145


## generate heatmap of top 100 differentially expressed genes

In [16]:
def generate_expression_heatmap(expression_df, volcano_plot_df, n=100, exclude_value=10):
    # Get the indices of the top n rows based on absolute values of 'logFC'
    # get the genes most differentially expressed (high log FC values)
    # if we exclude -exclude_value, exclude_value values
    top_n_indices = volcano_plot_df[
        (volcano_plot_df['logFC'].abs() != exclude_value)
    ]['logFC'].abs().nlargest(n).index
    
    top_n_rows = volcano_plot_df.loc[top_n_indices].set_index('gene')

    # go back to the expression df and make a heatmap of the top n genes
    expression_df_heatmap = expression_df.loc[top_n_rows.index]
    
    return expression_df_heatmap

heatmap_data = generate_expression_heatmap(expression_df, volcano_plot_df)


In [17]:
heatmap_data.head(3)

,RESPOND_10100218,RESPOND_10100291,RESPOND_10100412,RESPOND_10100478,RESPOND_10100596,RESPOND_10100615,RESPOND_10100801,RESPOND_10100884,RESPOND_10100899,RESPOND_10100952,...,RESPOND_80100242,RESPOND_80100313,RESPOND_80100345,RESPOND_80100411,RESPOND_80100526,RESPOND_80100556,RESPOND_80100590,RESPOND_81100031,RESPOND_40100842,RESPOND_80100259
gene,,,,,,,,,,,,,,,,,,,,,
TMEM238L,0.0,0.0,0.0,0.0,0.000012,0.0,0.000003,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000025,0.0
RN7SL321P,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
RNVU1-19,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.915644,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.471448,0.0


In [18]:
# can check mutation status of genes by checking individuals_mutated_target_gene

In [19]:
import pandas as pd

def get_mutated_status(expression_df_heatmap, individuals_mutated_target_gene, output_folder=target_gene):
    individuals_mutated_target_gene = list(individuals_mutated_target_gene)
    
    mutated_status = [1 if item in individuals_mutated_target_gene else 0 for item in expression_df_heatmap.columns]

    sample_categories_df = pd.DataFrame({
        'Sample': expression_df_heatmap.columns,
        'Mutation Status': mutated_status
    })

    output_filename = f'{output_folder}/sample_mutation_status.csv'
    print(output_filename)
    sample_categories_df.to_csv(output_filename,  index=False)
    return sample_categories_df

mutated_status_df = get_mutated_status(heatmap_data, individuals_mutated_target_gene)


RPL10/sample_mutation_status.csv


In [20]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.cluster.hierarchy import fcluster, linkage

def hierarchical_clustering(expression_df_heatmap, row_threshold=7, col_threshold=7, output_folder=target_gene):
    # Cluster the rows and columns using hierarchical clustering
    row_linkage = linkage(expression_df_heatmap, method='ward')
    col_linkage = linkage(expression_df_heatmap.T, method='ward')

    # Assign cluster labels using cluster
    row_clusters = fcluster(row_linkage, t=row_threshold, criterion='maxclust')
    col_clusters = fcluster(col_linkage, t=col_threshold, criterion='maxclust')

    # Create dictionaries to store cluster information
    row_cluster_info = {f"Cluster {cluster}": expression_df_heatmap.index[row_clusters == cluster] for cluster in np.unique(row_clusters)}
    col_cluster_info = {f"Cluster {cluster}": expression_df_heatmap.columns[col_clusters == cluster] for cluster in np.unique(col_clusters)}

    row_cluster_file = os.path.join(output_folder, 'row_clusters.txt')
    with open(row_cluster_file, 'w') as file:
        for cluster, items in row_cluster_info.items():
            file.write(f"{cluster}:\n")
            file.write(f"{', '.join(items)}\n\n")

    # Save column cluster information to a text file
    col_cluster_file = os.path.join(output_folder, 'col_clusters.txt')
    with open(col_cluster_file, 'w') as file:
        for cluster, items in col_cluster_info.items():
            file.write(f"{cluster}:\n")
            file.write(f"{', '.join(items)}\n\n")
    
    return row_linkage, col_linkage, row_cluster_info, col_cluster_info

# Example usage:
row_linkage, col_linkage, row_clusters, col_clusters = hierarchical_clustering(heatmap_data)


In [21]:
import seaborn as sns
import matplotlib.pyplot as plt

def create_clustered_heatmap_and_save(expression_df_heatmap, 
                                      row_linkage, 
                                      col_linkage, 
                                      sample_mutation_df, 
                                      output_folder=target_gene):
    # Group labels and colors for color bar
    type_map = {1: 'red', 0: 'yellow'}

    # Create a clustered heatmap
    clustered_df = sns.clustermap(
        expression_df_heatmap,
        row_linkage=row_linkage,
        col_linkage=col_linkage,
        cmap='viridis',
        annot=True,
        fmt=".1f",
        linewidths=.5,
        col_colors=sample_mutation_df.set_index('Sample')['Mutation Status'].map(type_map),
        standard_scale=0  # standardize by the rows
    )
    output_filename = f'{output_folder}/heatmap.png'
    
    # Save the plot to a file
    clustered_df.savefig(output_filename)

    # Close the plot to prevent displaying it in the notebook (optional)
    plt.close()

    return clustered_df

clustered_heatmap = create_clustered_heatmap_and_save(heatmap_data, 
                                                      row_linkage, 
                                                      col_linkage,
                                                      mutated_status_df)


In [22]:
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram

## add more args to the function 

def plot_and_save_dendrograms(row_linkage, col_linkage, expression_df_heatmap, output_folder=target_gene):
    # Plot the row dendrogram
    plt.figure(figsize=(12, 6))
    gene_plot = dendrogram(row_linkage, 
                           labels=expression_df_heatmap.index, 
                           truncate_mode='lastp', 
                           p=int(0.75 * len(row_linkage)), show_leaf_counts=True)
    plt.title('Row Dendrogram')
    
    # Save the row dendrogram plot to a file
    row_dendrogram_filename = f'{output_folder}/row_dendrogram.png'
    plt.savefig(row_dendrogram_filename)
    plt.close()

    # Plot the column dendrogram
    plt.figure(figsize=(12, 6))
    sample_plot = dendrogram(col_linkage, labels=expression_df_heatmap.columns, orientation='top', 
                             truncate_mode='lastp', p=int(0.75 * len(row_linkage)), show_leaf_counts=True)
    plt.title('Column Dendrogram')
    
    # Save the column dendrogram plot to a file
    col_dendrogram_filename = f'{output_folder}/col_dendrogram.png'
    plt.savefig(col_dendrogram_filename)
    plt.close()

    return gene_plot, sample_plot

row_dendrogram, col_dendrogram = plot_and_save_dendrograms(row_linkage, 
                                                           col_linkage, 
                                                           heatmap_data)


In [23]:
def histogram_of_column_and_save(df, column, output_folder=target_gene):
    # Plot the distribution of p-values
    plt.figure(figsize=(10, 6))
    plt.hist(df[column], bins=30, color='blue', edgecolor='black')
    plt.title(f'Distribution of {column}')
    plt.xlabel('P-values')
    plt.ylabel('Frequency')

    output_filename = f'{output_folder}/histogram_{column}.png'
    # Save the histogram plot to a file
    plt.savefig(output_filename)
    plt.close()

histogram_of_column_and_save(volcano_plot_df, 'adjusted_pvalue')


In [24]:
# clip logFC for the plot. not above bc need the full value for the histogram 
def volcano_plot(input_file_path, yaxis, xaxis, output_folder=target_gene):
    df = pd.read_csv(input_file_path)
    plt.scatter(x=df[xaxis],y=df[yaxis].apply(lambda x:-np.log10(x)),s=1)

    plt.xlabel(xaxis)
    plt.ylabel(f'-log10({yaxis})')
    plt.title('Volcano Plot')
    output_filename = f'{output_folder}/volcano_plot.png'
    plt.savefig(output_filename)
    plt.close()

In [25]:
def generate_expression_boxplots(expression_df, volcano_plot_df, n=10, exclude_value=10):
    # Get the indices of the top n rows based on absolute values of 'logFC'
    # get the genes most differentially expressed (high log FC values)
    # if we exclude -exclude_value, exclude_value values
    top_n_indices = volcano_plot_df[
        (volcano_plot_df['logFC'].abs() != exclude_value)
    ]['logFC'].abs().nlargest(n).index
    
    top_n_rows = volcano_plot_df.loc[top_n_indices].set_index('gene')

    # go back to the expression df and make a heatmap of the top n genes
    expression_df_heatmap = expression_df.loc[top_n_rows.index]
    
    return expression_df_heatmap

heatmap_data = generate_expression_heatmap(expression_df, volcano_plot_df)


In [26]:
volcano_plot(input_file_path=volcano_input_filename, yaxis='pvalue', xaxis='logFC')